In [43]:
import pandas as pd # type: ignore
import datetime
import calendar
import numpy as np # type: ignore
import re
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from sklearn.ensemble import IsolationForest

pd.set_option('future.no_silent_downcasting', True)

In [44]:
super_sheet = pd.ExcelFile(r"/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI/SSKI EKSTERNAL_25 Okt 2024.xlsx")

In [45]:
def prepare_dataframe(sheet_name, header_count):
    if sheet_name in ['11','16']:
        input_df = super_sheet.parse(sheet_name, header=[3, 2 + header_count])
    else:
        input_df = super_sheet.parse(sheet_name, header=[4, 3 + header_count])

    # Clean up and modify the column headers
    input_df.columns = input_df.columns.map(lambda x: str(x[0]).upper() if 'Unnamed' in str(x) else x)

    # Strip spaces in tuple columns
    input_df.columns = input_df.columns.map(lambda x: (x[0], x[1].strip()) if isinstance(x, tuple) and isinstance(x[1], str) else x)

    # Combine the multi-level column headers into a single string
    input_df.columns = input_df.columns.map(lambda x: f'{x[0]}-{x[1].capitalize()}' if isinstance(x, tuple) else x)

    first_column = input_df.iloc[:, 0]
    second_column = input_df.iloc[:, 1]

    # Filter based on the year (assuming year_to_check is a variable in scope)
    input_df = input_df.filter(regex=str(20))

    # Insert the 'NO' and 'Komponen' columns
    input_df.insert(0, 'NO', first_column)
    input_df.insert(1, 'Komponen', second_column)

    # Remove rows after the 'Keterangan' row (if it exists)
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]

    return input_df

df = prepare_dataframe("4", 2)
df

,NO,Komponen,2005,2006,2007,2008,2009,2010,2011,2012,...,2022-Q2,2022-Q3,2022-Q4,2023,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2
0,1,Core FSIs for Deposit takers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.,.
1,2,Regulatory capital to risk-weighted assets,2.018410e+01,2.125834e+01,1.952652e+01,1.663261e+01,1.715633e+01,1.583715e+01,1.524985e+01,1.616445e+01,...,2.310304e+01,2.352603e+01,2.413405e+01,2.584111e+01,2.321433e+01,2.525690e+01,2.547301e+01,2.584111e+01,24.2906,24.437104
2,3,Total regulatory capital,1.499328e+08,1.680040e+08,1.955812e+08,2.124505e+08,2.404399e+08,2.830622e+08,3.577378e+08,4.380935e+08,...,1.426490e+09,1.474245e+09,1.545912e+09,1.663298e+09,1.507131e+09,1.553433e+09,1.598943e+09,1.663298e+09,1604557738.256399,1651639225.5341
3,4,Risk-weighted assets,7.428265e+08,7.902968e+08,1.001618e+09,1.277313e+09,1.401464e+09,1.787330e+09,2.345845e+09,2.710228e+09,...,6.174467e+09,6.266440e+09,6.405521e+09,6.436634e+09,6.492244e+09,6.150531e+09,6.277008e+09,6.436634e+09,6605673526.403298,6758735617.138998
4,5,Tier 1 capital to risk-weighted assets,1.544958e+01,1.644884e+01,1.558629e+01,1.373710e+01,1.432014e+01,1.400932e+01,1.350456e+01,1.422485e+01,...,2.139775e+01,2.184782e+01,2.255920e+01,2.418378e+01,2.168986e+01,2.365053e+01,2.388114e+01,2.418378e+01,22.744696,22.931249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,212,Total gross loans,8.221774e+08,9.173799e+08,1.107123e+09,1.473261e+09,1.662246e+09,1.961254e+09,2.381914e+09,3.020937e+09,...,6.776339e+09,7.051318e+09,7.295716e+09,7.925380e+09,7.303767e+09,7.543825e+09,7.733890e+09,7.925380e+09,8000771521.443933,8155013645.16169
212,213,Commercial real estate loans to total gross loans,8.656033e-01,1.208650e+00,1.425522e+00,1.525244e+00,1.318092e+00,5.025799e+00,1.483590e+01,1.018863e+01,...,1.544998e+01,1.517310e+01,1.502278e+01,1.469755e+01,1.514551e+01,1.491263e+01,1.476139e+01,1.469755e+01,14.560103,14.999295
213,214,Commercial real estate loans,7.116795e+06,1.108791e+07,1.578228e+07,2.247082e+07,2.190993e+07,9.856868e+07,3.533785e+08,3.077921e+08,...,1.046943e+09,1.069903e+09,1.096020e+09,1.164837e+09,1.106193e+09,1.124982e+09,1.141630e+09,1.164837e+09,1164920582.555511,1223194550.859583
214,215,Total gross loans,8.221774e+08,9.173799e+08,1.107123e+09,1.473261e+09,1.662246e+09,1.961254e+09,2.381914e+09,3.020937e+09,...,6.776339e+09,7.051318e+09,7.295716e+09,7.925380e+09,7.303767e+09,7.543825e+09,7.733890e+09,7.925380e+09,8000771521.443933,8155013645.16169


In [46]:
def prepare_dataframe5(sheet_name):
    if sheet_name == '5a':
        header_array = [3, 4, 5]
    else:
        header_array = [2, 3, 4, 5]

    input_df = super_sheet.parse(sheet_name, header=header_array)

    input_df.columns = input_df.columns.map(lambda x: str(x[0]).upper() if 'Unnamed' in str(x) else x)

    input_df.columns = input_df.columns.map(
        lambda x: tuple(
            str(x[i]).upper() if i == 0 and 'Unnamed' in str(x[1]) else x[i].strip() if isinstance(x[i], str) else x[i]
            for i in range(len(x))
        ) if isinstance(x, tuple) else x
    )

    # Remove row with "keterangan" and rows following
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]
    
    # Gabungkan elemen kolom multi-level menjadi satu string secara fleksibel berdasarkan panjang tuple
    input_df.columns = input_df.columns.map(lambda x: '-'.join([str(i).capitalize() if isinstance(i, str) else str(i) for i in x]) if isinstance(x, tuple) else x)

    first_column = input_df.iloc[:, 0]
    second_column = input_df.iloc[:, 1]

    # Filter columns where the name contains a year (assuming year_to_check is a variable in scope)
    input_df = input_df.filter(regex=str(20))

    # Insert the 'NO' and 'Komponen' columns
    input_df.insert(0, 'NO', first_column)
    input_df.insert(1, 'Komponen', second_column)

    # Remove rows after the 'Keterangan' row (if it exists)
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]

    # Drop columns that contain only a year as their name (e.g., '2020', '2021')
    input_df = input_df.loc[:, ~input_df.columns.str.match(r'^\d{4}$')]

    return input_df

In [47]:
def split5a(df):
    claims_df = df.filter(like='Claims')
    liabilities_df = df.filter(like='Liabilities')
    common_columns = df.filter(regex='NO|Komponen')
    # Menggabungkan kolom Nomor dan Komponen ke DataFrame Claims dan Liabilities
    claims_df = pd.concat([common_columns, claims_df], axis=1)
    liabilities_df = pd.concat([common_columns, liabilities_df], axis=1)
    return claims_df, liabilities_df

def split5b(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...)
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_ld = pd.concat([common_columns, df.filter(like='Claims-Loan & deposits')], axis=1)
    liabilities_df_ld = pd.concat([common_columns, df.filter(like='Liabilities-Loan & deposits')], axis=1)
    claims_df_ds = pd.concat([common_columns, df.filter(like='Claims-Debt securities')], axis=1)
    liabilities_df_ds = pd.concat([common_columns, df.filter(like='Liabilities-Debt securities')], axis=1)
    claims_df_oi = pd.concat([common_columns, df.filter(like='Claims-Other instruments')], axis=1)
    liabilities_df_oi = pd.concat([common_columns, df.filter(like='Liabilities-Other instruments')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_ld, liabilities_df_ld, claims_df_ds, liabilities_df_ds, claims_df_oi, liabilities_df_oi

def split5d(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_b = pd.concat([common_columns, df.filter(like='Claims-Banks')], axis=1)
    liabilities_df_b = pd.concat([common_columns, df.filter(like='Liabilities-Banks')], axis=1)
    claims_df_nb = pd.concat([common_columns, df.filter(like='Claims-Nonbanks')], axis=1)
    liabilities_df_nb = pd.concat([common_columns, df.filter(like='Liabilities-Nonbanks')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_b, liabilities_df_b, claims_df_nb, liabilities_df_nb

def split5d1(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_r = pd.concat([common_columns, df.filter(like='Claims-Related offices')], axis=1)
    liabilities_df_r = pd.concat([common_columns, df.filter(like='Liabilities-Related offices')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r

def split5d2(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_nb = pd.concat([common_columns, df.filter(like='Claims-Non-bank financial ins.')], axis=1)
    liabilities_df_nb = pd.concat([common_columns, df.filter(like='Liabilities-Non-bank financial ins.')], axis=1)
    claims_df_nf = pd.concat([common_columns, df.filter(like='Claims-Non-financial corporations')], axis=1)
    liabilities_df_nf = pd.concat([common_columns, df.filter(like='Liabilities-Non-financial corporations')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf

In [48]:
def prepare_all(name, input_df):

    # Step 2: Filter the DataFrame based on the selected columns
    filtered_df = input_df

    # Step 3: Replace '-' with NaN and drop rows with NaN
    filtered_df.replace('-', np.nan, inplace=True)

    # Step 4: Transpose the table, so NO and Komponen become the headers
    transposed_df = filtered_df.set_index(['NO', 'Komponen']).T
    
    return transposed_df

In [49]:
import numpy as np
import pandas as pd

# Month mapping dictionary
month_mapping = {
    'Jan': 'Jan',
    'Feb': 'Feb',
    'Mar': 'Mar',
    'Apr': 'Apr',
    'Mei': 'May',
    'Jun': 'Jun',
    'Jul': 'Jul',
    'Agt': 'Aug',
    'Sept': 'Sep',
    'Okt': 'Oct',
    'Nov': 'Nov',
    'Des': 'Dec',
    'Q1': 'Mar',
    'Q2': 'Jun',
    'Q3': 'Sep',
    'Q4': 'Dec'
}

# Function to reformat Periode
def reformat_periode(periode):
    tokens = periode.split('-')
    
    if len(tokens) > 1:
        year = tokens[0]
        month = tokens[1]
        # Clean the month in case there are additional characters like 'Dec*' or 'OctX'
        month = re.sub(r'[^A-Za-z0-9]', '', month)
        
        # Map the cleaned month to the dictionary
        return f"{year}-{month_mapping.get(month, month)}"
    
    elif len(tokens) == 1:
        # Single-token period handling
        cleaned_token = re.sub(r'[^A-Za-z0-9]', '', tokens[0])  # Clean the single token
        return month_mapping.get(cleaned_token, cleaned_token)
    
    return periode  # Handle unexpected cases gracefully

def detect_outliers_iqr(name,df):
    # Pastikan DataFrame tidak kosong sebelum melanjutkan
    if df.empty:
        print("DataFrame is empty.")
        return pd.DataFrame(columns=['Komponen','Detail', 'Tahun', 'Nilai', 'Batas Bawah', 'Batas Atas', 'Outlier', 'Gap', 'last_period'])
    
    if name == "5.d.2":
        name = "5d2"
    
    # Replace string values (including '-') with NaN
    df = df.replace('-', np.nan).map(lambda x: np.nan if isinstance(x, str) else x)
    
    # Get the last period for each component (assuming index represents period/year)
    last_period = df.index[-1] if len(df.index) > 0 else None

    outliers_data = []
    for column in df.columns:
        # Ensure all values in the column are numeric
        df[column] = pd.to_numeric(df[column], errors='coerce')
        col_data = df[column].dropna()
        
        if col_data.empty:
            continue  # Skip columns with no numeric data
        
        # Calculate IQR, lower bound, and upper bound
        Q1 = col_data.quantile(0.25)
        Q3 = col_data.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Process each value and identify if it's an outlier
        for index, value in df[column].items():
            is_outlier = value < lower_bound or value > upper_bound
            
            # Calculate gap only for outliers, leave empty for non-outliers
            gap = None
            if is_outlier:
                gap = (value - upper_bound) if value > upper_bound else (lower_bound - value)
            token = index.split("-",2)
            if len(token) == 3:
                detail = token[2]
            else:
                detail = '-'


            outliers_data.append({
                'Tabel' : name,
                'No_Komponen':int(column[0]),
                'Komponen': column[1],
                'Detail' : detail,
                'Periode': index,
                'Nilai': value,
                'Batas Bawah': lower_bound,
                'Batas Atas': upper_bound,
                'Outlier': 'Ya' if is_outlier else 'Tidak',
                'Gap': gap,
                'last_period': last_period
            })
    
    # Create the DataFrame with the collected outlier data
    df_outliers = pd.DataFrame(outliers_data)

    # # Apply the function to the Periode column
    df_outliers['Periode'] = df_outliers['Periode'].apply(reformat_periode)
    df_outliers['last_period'] = df_outliers['last_period'].apply(reformat_periode)

    
    return df_outliers

In [50]:
# name = "1"
# raw_df = prepare_dataframe(name, 2)
# checked_df = prepare_all(name, raw_df)
# checked_df = checked_df.iloc[1:]
# outliers_df = detect_outliers_iqr(name,checked_df)
# # Add a new column 'No.' that contains the index values
# outliers_df.insert(0, 'No.', range(1, len(outliers_df) + 1))
# # Display the updated DataFrame
# display(outliers_df)

# output_path = r"/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/application/data/data_pencilan_sski.csv"
# outliers_df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd

sheet_names = ["1","2","3","4","5a","5b","5c","5d","5d.1","5.d.2","6","7","8","9","10","11","11a","12","13","14","15","16","16a","17","18","19","20"]  

all_outliers = []

# Loop to process all sheets
for name in sheet_names:
    print(f"Processing SSKI {name}...")

    if name == "5a":
        raw_df = prepare_dataframe5(name)
        
        claims_df, liabilities_df = split5a(raw_df)
        
        claims_df = prepare_all(name, claims_df)
        liabilities_df = prepare_all(name, liabilities_df)
        
        claims_outliers = detect_outliers_iqr(name, claims_df)
        all_outliers.append(claims_outliers)

        liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
        all_outliers.append(liabilities_outliers)

    elif name in ["5b", "5c"]:
        raw_df = prepare_dataframe5(name)
        
        splits = split5b(raw_df)
        
        for claims_df, liabilities_df in zip(splits[::2], splits[1::2]):
            claims_df = prepare_all(name, claims_df)
            claims_outliers = detect_outliers_iqr(name, claims_df)
            all_outliers.append(claims_outliers)

            liabilities_df = prepare_all(name, liabilities_df)
            liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
            all_outliers.append(liabilities_outliers)

    elif name == "5d":
        raw_df = prepare_dataframe5(name)
        
        splits = split5d(raw_df)
        
        for claims_df, liabilities_df in zip(splits[::2], splits[1::2]):
            claims_df = prepare_all(name, claims_df)
            claims_outliers = detect_outliers_iqr(name, claims_df)
            all_outliers.append(claims_outliers)

            liabilities_df = prepare_all(name, liabilities_df)
            liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
            all_outliers.append(liabilities_outliers)

    elif name == "5d.1":
        raw_df = prepare_dataframe5(name)
        
        claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r = split5d1(raw_df)
        
        for df in [claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r]:
            df = prepare_all(name, df)
            outliers = detect_outliers_iqr(name, df)
            all_outliers.append(outliers)

    elif name == "5.d.2":
        raw_df = prepare_dataframe5(name)
        
        claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf = split5d2(raw_df)
        
        for df in [claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf]:
            df = prepare_all(name, df)
            outliers = detect_outliers_iqr(name, df)
            all_outliers.append(outliers)

    else:
        raw_df = prepare_dataframe(name, 2)
        checked_df = prepare_all(name, raw_df)
        checked_df = checked_df.iloc[1:]
        outliers_df = detect_outliers_iqr(name, checked_df)
        all_outliers.append(outliers_df)

# Concatenate all outliers and export to a single CSV file
combined_outliers = pd.concat(all_outliers, ignore_index=True)
combined_outliers.insert(0, 'No.', range(1, len(combined_outliers) + 1))

combined_outliers.to_csv('/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/data/data_pencilan_sski.csv', index=False, sep=";")

Processing SSKI 1...
Processing SSKI 2...
Processing SSKI 3...
Processing SSKI 4...
Processing SSKI 5a...
Processing SSKI 5b...
Processing SSKI 5c...
Processing SSKI 5d...
Processing SSKI 5d.1...
Processing SSKI 5.d.2...
Processing SSKI 6...
Processing SSKI 7...
Processing SSKI 8...
Processing SSKI 9...
Processing SSKI 10...
Processing SSKI 11...
Processing SSKI 11a...
Processing SSKI 12...
Processing SSKI 13...
Processing SSKI 14...
Processing SSKI 15...
Processing SSKI 16...
Processing SSKI 16a...
Processing SSKI 17...
Processing SSKI 18...
Processing SSKI 19...
Processing SSKI 20...
